In [150]:
import numpy as np
import pandas as pd
import os
import time
import schedule
import joblib
import datetime
import multiprocessing as mp
from get_strategy_meta import get_all_strategy_meta
from query_api import get_path_dict, gen_connection, load_item_by_key, query_mysql
from get_signal_mongo import write_signal_mongo, get_signal_mongo
from get_signal_mysql import write_signal, get_signal
from mongo_functions import get_portfolio_db

# Check Log

In [162]:
db_name = "MLLog"
coll_name = "log"
db = get_portfolio_db(db_name)
coll = db[coll_name]
# r = coll.find({"code":"W006", "key":"s_AUDUSD"})
# r = coll.find({"code": "X004", "key": "s_TLT"})
cond = {"idd.code": "Y001"}
cond = {}
r = coll.find(cond)
df = pd.DataFrame([j for j in r])
# df.sort_values("time")000.6 
df.tail(50)

,Date,_id,idd,message,other
204327,2019-07-10 03:31:08.368,5d25943cc1689d062007814d,"{'code': 'Z001', 'key': 's_XAGUSD'}",Loaded: MINUTE_INDEX for XAGUSD,
204328,2019-07-10 03:31:08.369,5d25943cc1689d062007814e,"{'code': 'Z001', 'key': 's_XAGUSD'}",Using Mongo: 2019-07-06 03:30:00 - 2019-07-10 ...,
204329,2019-07-10 03:31:18.775,5d259446c1689d0620078150,"{'code': 'Z001', 'key': 's_XAGUSD'}",Scaling: Done!,
204330,2019-07-10 03:31:18.777,5d259446c1689d0620078151,"{'code': 'Z001', 'key': 's_XAGUSD'}",TimeseriesGenerator: Done!,
204331,2019-07-10 03:31:22.853,5d25944ac1689d0620078152,"{'code': 'Z001', 'key': 's_XAGUSD'}",Model Loaded!,
204332,2019-07-10 03:31:23.759,5d25944bc1689d0620078153,"{'code': 'Z001', 'key': 's_XAGUSD'}",Predict: Done!,
204333,2019-07-10 03:31:23.760,5d25944bc1689d0620078154,"{'code': 'Z001', 'key': 's_XAGUSD'}",s_XAGUSD,"{'last_date': 2019-07-10 03:30:00, 'query_date..."
204334,2019-07-10 03:31:24.248,5d25944c6e66cbc56007814c,"{'code': 'Z007', 'key': 'b_EURJPY'}","Loaded: scaler, DATA_PARAMS, MODEL_PARAMS, model.",
204335,2019-07-10 03:31:24.269,5d25944c6e66cbc56007814d,"{'code': 'Z007', 'key': 'b_EURJPY'}",Loaded: MINUTE_INDEX for EURJPY,
204336,2019-07-10 03:31:24.270,5d25944c6e66cbc56007814e,"{'code': 'Z007', 'key': 'b_EURJPY'}",Using Mongo: 2019-07-05 23:30:00 - 2019-07-10 ...,


In [72]:
df.head()

,_id,code,key,last_date,query_date,status,status_code,timeframe,write_time,y
0,5d255d4aa018d5cd702bb606,Y001,b_EMB,2019-07-09 16:00:00,2019-07-09 23:33:43.746,Outdated Signal.,-2,15,2019-07-09 23:36:42.174,1.087600e-01
1,5d255d56e8c45a3f092bb606,Y001,b_NZDUSD,2019-07-09 23:45:00,2019-07-09 23:33:43.746,Successful.,1,15,2019-07-09 23:36:54.225,5.551494e-01
2,5d255d5e3a8e8f52fc2bb606,Y001,b_XBI,2019-07-09 16:00:00,2019-07-09 23:33:43.746,Outdated Signal.,-2,15,2019-07-09 23:37:02.366,1.252234e-07
3,5d255d6690477923512bb606,Y001,b_XLE,2019-07-09 16:00:00,2019-07-09 23:33:43.746,Outdated Signal.,-2,15,2019-07-09 23:37:10.712,1.258450e-02
4,5d255d6e9c410022012bb606,Y001,s_EMB,2019-07-09 16:00:00,2019-07-09 23:33:43.746,Outdated Signal.,-2,15,2019-07-09 23:37:18.265,6.254176e-03


# Check Signal

In [101]:
db_name = "MLProduction"
coll_name = "SignalOneT1"
db = get_portfolio_db(db_name)
coll = db[coll_name]
# r = coll.find({"code":"W006", "key":"s_AUDUSD"})
# r = coll.find({"code": "X004", "key": "s_TLT"})
# cond = {"code": "W006", "key": "b_USDCAD"}
cond = {}
r = coll.find(cond)
df = pd.DataFrame([j for j in r])
df.head(20)


,_id,code,key,last_date,query_date,status,status_code,timeframe,write_time,y
0,5d2579f7a2965d7424a11f6c,Z_C007,s_AUDUSD,2019-07-10 01:40:00,2019-07-10 01:38:48.916,Successful.,1,5,2019-07-10 01:39:03.571,4.359218e-01
1,5d257a06c9ff148536a11f6c,Z_C007,s_NZDUSD,2019-07-10 01:40:00,2019-07-10 01:38:48.916,Successful.,1,5,2019-07-10 01:39:18.352,3.843376e-01
2,5d257a15486892db08a11f6c,Z_C007,s_XAUUSD,2019-07-10 01:40:00,2019-07-10 01:38:48.916,Successful.,1,5,2019-07-10 01:39:33.136,4.414816e-01
3,5d257a240fe8971c94a11f6b,Z001,b_XAGUSD,2019-07-10 01:40:00,2019-07-10 01:38:48.916,Successful.,1,5,2019-07-10 01:39:48.188,4.604424e-01
4,5d257a331d889f74eaa11f6b,Z001,s_XAGUSD,2019-07-10 01:40:00,2019-07-10 01:38:48.916,Successful.,1,5,2019-07-10 01:40:03.333,2.810065e-01
5,5d257a426ed96b7df5a11f6b,Z007,b_EURJPY,2019-07-10 01:40:00,2019-07-10 01:38:48.916,Successful.,1,5,2019-07-10 01:40:18.133,2.403364e-01
6,5d257a5039bf211543a11f6b,Z007,b_NZDUSD,2019-07-10 01:40:00,2019-07-10 01:38:48.916,Successful.,1,5,2019-07-10 01:40:32.757,3.920986e-01
7,5d257a5fc09b189fbda11f6b,Z007,b_XAGUSD,2019-07-10 01:40:00,2019-07-10 01:38:48.916,Successful.,1,5,2019-07-10 01:40:47.346,5.284283e-01
8,5d257a6d6f8bfdc82fa11f6b,Z007,s_EURJPY,2019-07-10 01:40:00,2019-07-10 01:38:48.916,Successful.,1,5,2019-07-10 01:41:01.762,6.638547e-02
9,5d257a7d21640a16dfa11f6b,Z007,s_NZDUSD,2019-07-10 01:40:00,2019-07-10 01:38:48.916,Successful.,1,5,2019-07-10 01:41:17.027,1.827652e-01


# Check Signal (Windows)

In [163]:
db_name = "MLProduction"
coll_name = "SignalOneLiveU1"
db = get_portfolio_db(db_name)
coll = db[coll_name]
# r = coll.find({"code":"W006", "key":"s_AUDUSD"})
# r = coll.find({"code": "X004", "key": "s_TLT"})
cond = {"code": "Y001"}
cond = {"code": "W006", "key": "b_USDCAD"}
cond = {}
r = coll.find(cond)
df = pd.DataFrame([j for j in r])
df.tail(10)

,_id,code,key,last_date,query_date,status,status_code,timeframe,write_time,y
53,5d25948d489688790c078155,Z007,s_EURJPY,2019-07-10 03:30:00,2019-07-10 03:30:00,Successful.,1,5,2019-07-10 03:32:29.638,1.378033e-01
54,5d25949f7e2a0bde3b078155,Z007,s_NZDUSD,2019-07-10 03:30:00,2019-07-10 03:30:00,Successful.,1,5,2019-07-10 03:32:47.839,7.681643e-01
55,5d2594b251a2d66340078155,Z007,s_XAGUSD,2019-07-10 03:30:00,2019-07-10 03:30:00,Successful.,1,5,2019-07-10 03:33:06.396,5.757349e-01
56,5d2594bbac26973942078155,Y001,b_EMB,2019-07-09 16:00:00,2019-07-10 03:30:00,Outdated Signal.,-2,15,2019-07-10 03:33:15.687,1.087600e-01
57,5d2594c9af75c569ae078155,Y001,b_NZDUSD,2019-07-10 03:30:00,2019-07-10 03:30:00,Successful.,1,15,2019-07-10 03:33:29.331,1.810268e-01
58,5d2594d258efea2d6e078155,Y001,b_XBI,2019-07-09 16:00:00,2019-07-10 03:30:00,Outdated Signal.,-2,15,2019-07-10 03:33:38.642,1.252234e-07
59,5d2594dba838b03bcb078155,Y001,b_XLE,2019-07-09 16:00:00,2019-07-10 03:30:00,Outdated Signal.,-2,15,2019-07-10 03:33:47.834,1.258450e-02
60,5d2594e54924ec2a5f078155,Y001,s_EMB,2019-07-09 16:00:00,2019-07-10 03:30:00,Outdated Signal.,-2,15,2019-07-10 03:33:57.938,6.254176e-03
61,5d2594f479b9dc64be078155,Y001,s_NZDUSD,2019-07-10 03:30:00,2019-07-10 03:30:00,Successful.,1,15,2019-07-10 03:34:12.021,7.126663e-01
62,5d2594fdfa77f3b754078155,Y001,s_XBI,2019-07-09 16:00:00,2019-07-10 03:30:00,Outdated Signal.,-2,15,2019-07-10 03:34:21.973,6.518391e-01


In [166]:
df["dd"] = [j.total_seconds()/60 for j in (df["write_time"] - df["query_date"])]
df.sort_values(["dd", "timeframe"], ascending = False)

,_id,code,key,last_date,query_date,status,status_code,timeframe,write_time,y,dd
22,5d2591b06bf06a5bf2078155,Y002,s_XAGUSD,2019-07-10 03:15:00,2019-07-10 03:15:00,Successful.,1,15,2019-07-10 03:20:16.354,5.580189e-01,5.272567
21,5d2591a2d8cccbd721078155,Y002,s_TBT,2019-07-09 16:00:00,2019-07-10 03:15:00,Outdated Signal.,-2,15,2019-07-10 03:20:02.875,6.724984e-01,5.047917
20,5d2591993936249767078155,Y002,b_XAGUSD,2019-07-10 03:15:00,2019-07-10 03:15:00,Successful.,1,15,2019-07-10 03:19:53.642,3.524624e-01,4.894033
19,5d25918bb4478ace50078155,Y002,b_TBT,2019-07-09 16:00:00,2019-07-10 03:15:00,Outdated Signal.,-2,15,2019-07-10 03:19:39.925,4.180613e-01,4.665417
18,5d259183daa5d63c22078155,Y001,s_XLE,2019-07-09 16:00:00,2019-07-10 03:15:00,Outdated Signal.,-2,15,2019-07-10 03:19:31.561,3.284287e-03,4.526017
17,5d25917bc55aea959c078155,Y001,s_XBI,2019-07-09 16:00:00,2019-07-10 03:15:00,Outdated Signal.,-2,15,2019-07-10 03:19:23.437,6.518391e-01,4.390617
62,5d2594fdfa77f3b754078155,Y001,s_XBI,2019-07-09 16:00:00,2019-07-10 03:30:00,Outdated Signal.,-2,15,2019-07-10 03:34:21.973,6.518391e-01,4.366217
16,5d25917351f833c1fe078155,Y001,s_NZDUSD,2019-07-10 03:15:00,2019-07-10 03:15:00,Successful.,1,15,2019-07-10 03:19:15.215,7.192149e-01,4.253583
61,5d2594f479b9dc64be078155,Y001,s_NZDUSD,2019-07-10 03:30:00,2019-07-10 03:30:00,Successful.,1,15,2019-07-10 03:34:12.021,7.126663e-01,4.200350
15,5d25916624be011c32078155,Y001,s_EMB,2019-07-09 16:00:00,2019-07-10 03:15:00,Outdated Signal.,-2,15,2019-07-10 03:19:02.161,6.254176e-03,4.036017


In [154]:
# df.set_index("query_date")["y"].plot()

# Check prices

In [ ]:
# %%time
db_name = "Production"
coll_name = "prices"
db = get_portfolio_db(db_name)
coll = db[coll_name]
name_list = coll.distinct("name")
r = coll.find({"name": "EFA"})
df = pd.DataFrame([j for j in r])
df = df.set_index("time")
df.index.is_monotonic_increasing
df.head()
df.tail()
df["price"].plot()
dp_list = []
for name in name_list:
    r = coll.find({"name": name})
    df = pd.DataFrame([j for j in r])
    df = df.set_index("time")
    print(name, df.index.max()- df.index.min())
    if df.index.duplicated().any():
        dp_list.append(name)
    
print(len(name_list))
",".join(dp_list)

In [37]:
db_name = "Production"
coll_name = "prices"
db = get_portfolio_db(db_name)
coll = db[coll_name]
r = coll.find({"name": "IWM"})
pdf = pd.DataFrame([j for j in r])
pdf
# pdf = pdf.set_index("time")
# d = pdf.index[pdf.index.duplicated()]
# pdf.loc[d]



""


In [ ]:
pdf.index.duplicated()

In [ ]:
df.set_index("query_date")["status_code"].plot()

In [108]:
timeframe = 5

In [112]:
price_db ="Production"
price_coll = "prices"
d2 = datetime.datetime.now()
d1 = d2 - datetime.timedelta(1,0,0)
prod_db = get_portfolio_db(price_db)
coll = prod_db[price_coll]
result = coll.find({"time": {"$gte": d1, "$lte": d2}})
price_df = pd.DataFrame([j for j in result])
price_df["name"] = price_df["name"].str.replace('@', '')
price_df = price_df.pivot(index='time', columns='name', values='price')
price_df = price_df.reset_index()
price_df = price_df.rename(columns={"time": "Date"})
price_df = price_df.set_index("Date")

if not price_df.index.is_monotonic_increasing:
    price_df = price_df.sort_index(ascending=True)

price_df = price_df.resample("{}min".format(timeframe), how = "last",label='right', closed = "right")
print(price_df.head(5))
print(price_df.tail(5))

name                 ACWX  AGG  ASHR   AUDUSD  DBA  DBC  DIA  EEM  EFA  EMB  \
Date                                                                          
2019-07-09 02:45:00   NaN  NaN   NaN  0.69579  NaN  NaN  NaN  NaN  NaN  NaN   
2019-07-09 02:50:00   NaN  NaN   NaN  0.69580  NaN  NaN  NaN  NaN  NaN  NaN   
2019-07-09 02:55:00   NaN  NaN   NaN  0.69559  NaN  NaN  NaN  NaN  NaN  NaN   
2019-07-09 03:00:00   NaN  NaN   NaN  0.69560  NaN  NaN  NaN  NaN  NaN  NaN   
2019-07-09 03:05:00   NaN  NaN   NaN  0.69561  NaN  NaN  NaN  NaN  NaN  NaN   

name                 ...  XLI  XLK  XLP  XLU  XLV  XLY  XME  XOP  XRT  \
Date                 ...                                                
2019-07-09 02:45:00  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2019-07-09 02:50:00  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2019-07-09 02:55:00  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2019-07-09 03:00:00  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2019-07-

In [115]:
prod_db = get_portfolio_db(price_db)
coll = prod_db[price_coll]
result = coll.find({"time": {"$gte": d1, "$lte": d2}})
price_df2 = pd.DataFrame([j for j in result])
price_df2.tail()

,_id,name,price,time
25677,5d2551d831353b07c04e1529,ZB 09-19,154.68750,2019-07-10 02:37:00
25678,5d25525031353b07c04e1533,ZB 09-19,154.68750,2019-07-10 02:38:00
25679,5d25527431353b07c04e1542,ZB 09-19,154.71875,2019-07-10 02:40:00
25680,5d2552c631353b07c04e154c,ZB 09-19,154.68750,2019-07-10 02:41:00
25681,5d2552f431353b07c04e1551,ZB 09-19,154.68750,2019-07-10 02:42:00


In [138]:
d2.day

10

In [139]:
corrected_minute = d2.minute - d2.minute % 15
query_date = datetime.datetime(d2.year, d2.month, d2.day, d2.hour, corrected_minute)

datetime.datetime(2019, 7, 10, 2, 30)